In [7]:
import mysql.connector
from datetime import datetime
import mysql.connector
import pandas as pd
import warnings
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import gym
import numpy as np
import torch 
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
import torch.nn.functional as F

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [8]:
def get_table_names(host, user, password, database):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query to retrieve table names
        query = f"SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{database}'"
        cursor.execute(query)

        # Fetch all the rows
        tables = cursor.fetchall()

        # Print table names
        # for (table_name,) in tables:
        #     print(table_name)

    except mysql.connector.Error as error:
        print(f"Error: {error}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            return tables

In [9]:
def table_to_dataframe(host, user, password, database, table_name):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Use backticks around the table name in the query
        query = f"SELECT * FROM `{table_name}`"

        # Use pandas to read the query into a DataFrame
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            df = pd.read_sql(query, conn)

        return df

    except mysql.connector.Error as error:
        print(f"Error: {error}")
        return None
    finally:
        if conn.is_connected():
            conn.close()

In [10]:
# databaseA = "A" + datetime.now().strftime("%Y%m%d") 
databaseA = 'a20240112'
password = '93150lbm!!'

tables = get_table_names('127.0.0.1', 'root', password, databaseA)

tables = [code for (code,) in tables]
print(tables)

['000440', '005290', '009620', '009730', '013990', '021080', '024060', '025320', '027360', '027830', '029480', '033540', '036540', '036630', '038870', '041020', '041190', '041460', '047310', '048770', '049080', '050890', '053050', '056080', '057680', '058610', '059100', '060250', '060280', '060310', '064260', '065350', '065450', '066790', '068760', '072950', '080220', '080530', '085670', '086960', '088800', '090150', '090360', '090710', '094170', '094480', '096630', '098460', '099440', '104480', '108490', '115160', '117730', '129890', '136480', '142210', '203650', '205100', '205470', '207760', '208340', '214270', '214680', '215100', '218150', '227100', '228760', '232140', '241820', '253590', '254120', '264850', '270520', '270660', '277070', '290690', '291230', '293580', '299900', '306620', '307930', '317830', '319400', '321260', '321370', '348340', '356680', '380540', '382800', '383930', '388790', '391710', '403870', '417200', '419050', '419270', '450520', '452190', '452300', '900250']

In [11]:
code = tables[0]

df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)
df['체결시간'] = pd.to_datetime(df['체결시간'])
df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
df['거래대금'] = df['현재가'] * df['거래량']
df = df.sort_values(by='체결시간', ignore_index=True)
df['time_diff'] = df['체결시간'].diff()
df['time_diff_seconds'] = df['time_diff'].dt.total_seconds()

df


,id,체결시간,현재가,거래량,거래대금,time_diff,time_diff_seconds
0,1,2024-01-12 09:41:33.012765,18750,39,731250,NaT,NaN
1,2,2024-01-12 09:41:36.405014,18740,-5,-93700,0 days 00:00:03.392249,3.392249
2,3,2024-01-12 09:41:37.399747,18740,1,18740,0 days 00:00:00.994733,0.994733
3,4,2024-01-12 09:41:43.996522,18730,-146,-2734580,0 days 00:00:06.596775,6.596775
4,5,2024-01-12 09:41:46.801512,18740,20,374800,0 days 00:00:02.804990,2.804990
...,...,...,...,...,...,...,...
8896,8895,2024-01-12 15:19:52.595263,18960,303,5744880,0 days 00:00:00.001522,0.001522
8897,8898,2024-01-12 15:19:55.805188,18920,-30,-567600,0 days 00:00:03.209925,3.209925
8898,8899,2024-01-12 15:19:57.598681,18960,1,18960,0 days 00:00:01.793493,1.793493
8899,8900,2024-01-12 15:30:25.579565,19090,7559,144301310,0 days 00:10:27.980884,627.980884


In [12]:
df_use = df[['거래대금', 'time_diff_seconds', '현재가']]
df_use

,거래대금,time_diff_seconds,현재가
0,731250,NaN,18750
1,-93700,3.392249,18740
2,18740,0.994733,18740
3,-2734580,6.596775,18730
4,374800,2.804990,18740
...,...,...,...
8896,5744880,0.001522,18960
8897,-567600,3.209925,18920
8898,18960,1.793493,18960
8899,144301310,627.980884,19090


In [13]:
import gym
import numpy as np

class TradingEnv(gym.Env):
    def __init__(self, df, window_size=50, initial_budget=10000000):
        super(TradingEnv, self).__init__()
        self.df = df.dropna()
        self.window_size = window_size
        self.initial_budget = initial_budget
        self.current_budget = initial_budget
        self.action_space = gym.spaces.Discrete(3)  # ['buy', 'hold', 'sell']
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(window_size, 3), dtype=np.float32)
        self.holding_stock = False
        self.stock_amount = 0
        self.buy_price = 0
        self.current_step = window_size
        self.holding_status_history = [0] * window_size  # Initialize with no stock held
        self.invalid_action_penalty = -0.0001
        self.prev_price = 0 


    def update_holding_status(self):
        """
        Update the holding status history.
        Add 1 if holding stock, 0 otherwise.
        """
        if len(self.holding_status_history) >= self.window_size:
            self.holding_status_history.pop(0)  # Remove the oldest status
        self.holding_status_history.append(1 if self.holding_stock else 0)

    def sell_all_stocks(self):
        """
        Sell all held stocks at the current price.
        This method is called at the end of each episode.
        """
        if self.holding_stock:
            current_price = self.df.iloc[self.current_step, 2]  # 현재가
            sell_amount = self.stock_amount * current_price
            self.current_budget += sell_amount
            self.stock_amount = 0
            self.holding_stock = False
            
    def step(self, action):
        self.reward = 0
        self.done = False
        current_price = self.df.iloc[self.current_step, 2]  # 현재가

        if action == 0 : #buy 
            if self.holding_stock:
                self.reward = self.invalid_action_penalty
            else : 
                self.stock_amount = self.current_budget // current_price
                if self.stock_amount > 0:
                    self.holding_stock = True
                    self.buy_price = current_price
                    self.current_budget -= self.stock_amount * current_price
                    
        elif action == 1 and self.holding_stock : # hold
            
            self.reward = (current_price - self.buy_price) / self.buy_price 
            


        elif action == 2 :  # Sell
            if not self.holding_stock:
                self.reward = self.invalid_action_penalty
            else :
                self.holding_stock = False
                sell_amount = self.stock_amount * current_price
                self.reward = (sell_amount - self.buy_price * self.stock_amount) / (self.buy_price * self.stock_amount) 
                self.current_budget += sell_amount
                self.stock_amount = 0

        self.update_holding_status()

        if self.current_step >= len(self.df)-1:
            self.done = True  # End the episode when there are no more steps
            self.sell_all_stocks() 
        else : 
            self.current_step += 1

        self.prev_price = current_price
        
        return self._next_observation(), self.reward, self.done, {}

    def reset(self):
        self.current_step = self.window_size
        self.current_budget = self.initial_budget
        self.holding_stock = False
        self.stock_amount = 0
        self.buy_price = 0
        self.holding_status_history = [0] * self.window_size

        return self._next_observation()

    def _next_observation(self):
        start = self.current_step - self.window_size
        end = self.current_step
        obs = self.df.iloc[start:end, [0, 1]].values
        
        holding_status = np.array(self.holding_status_history).reshape(-1, 1)
        obs_with_holding_status = np.hstack((obs, holding_status))
        return obs_with_holding_status

    # Additional methods as needed


In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x


In [15]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, num_classes, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model)
        self.encoder = nn.Linear(input_dim, d_model)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout),
            num_layers=num_layers
        )
        self.decoder = nn.Linear(d_model, num_classes)

    def forward(self, src):
        src = self.encoder(src)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output.mean(dim=1))
        return output


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerPolicy(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, num_actions):
        super(TransformerPolicy, self).__init__()
        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = nn.Embedding(50, d_model)  # Assuming max length 50
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, num_actions)

    def forward(self, src):
        src = self.encoder(src)
        position = torch.arange(0, src.size(1), device=src.device).unsqueeze(0).repeat(src.size(0), 1)
        pos_encoded = self.pos_encoder(position)
        src += pos_encoded
        output = self.transformer_encoder(src)
        logits = self.decoder(output.mean(dim=1))
        stabilized_logits = logits - torch.max(logits, dim=1, keepdim=True).values
        return F.softmax(stabilized_logits, dim=-1)




In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimplePolicy(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimplePolicy, self).__init__()
        # Define the architecture
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(self.fc3(x), dim=-1)




In [22]:
# def process_observation(observation):
#     """
#     Process the observation for the model.
#     """
#     # Check if the observation is already a tensor
#     if isinstance(observation, torch.Tensor):
#         # If it's a tensor, just ensure it's on the correct device
#         return observation.to(device)
#     else:
#         # If it's not a tensor (e.g., numpy array or list), convert it to a tensor and move to the device
#         return torch.tensor(observation, dtype=torch.float32).to(device)

# def process_observation(observation):
#     """
#     Process the observation for the model.
#     """
#     # Flatten the observation
#     flattened_obs = observation.flatten()

#     # Check if the observation is already a tensor
#     if isinstance(flattened_obs, torch.Tensor):
#         # If it's a tensor, just ensure it's on the correct device
#         return flattened_obs.to(device)
#     else:
#         # If it's not a tensor (e.g., numpy array or list), convert it to a tensor and move to the device
#         return torch.tensor(flattened_obs, dtype=torch.float32).to(device)

def process_observation(observation):
    # Assuming observation is a flat vector of features
    # Reshape or process it to match [sequence_length, embed_size]
    processed_obs = observation.view(-1, embed_size)  # Adjust as needed
    return processed_obs.to(device)

def select_action(model, state):
    state_tensor = process_observation(state)
    state_tensor = state_tensor.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        action_probs = model(state_tensor)
    action = torch.multinomial(action_probs, 1).item()
    return action



# def select_action(model, state):
#     # Convert state to a tensor if it's a numpy array or a list
#     if isinstance(state, np.ndarray) or isinstance(state, list):
#         state_tensor = torch.FloatTensor(state).to(device)
#     elif isinstance(state, torch.Tensor):
#         # If it's already a tensor, ensure it's on the correct device
#         state_tensor = state.to(device)
#     else:
#         raise TypeError("Unrecognized type for state. Expected numpy array, list, or torch.Tensor")

#     state_tensor = state_tensor.unsqueeze(0)  # Add a batch dimension
#     with torch.no_grad():
#         action_probs = model(state_tensor)
#         print("Action probabilities:", action_probs)
#     action = torch.multinomial(action_probs, 1).item()
#     return action

# def select_action(model, state):
#     state_tensor = process_observation(state)
#     with torch.no_grad():
#         action_probs = model(state_tensor)
#     action = torch.multinomial(action_probs, 1).item()
#     return action




def compute_returns(rewards, gamma=0.99):
    """
    Compute discounted returns.
    """
    R = 0
    returns = []
    for r in reversed(rewards):
        R = r + gamma * R
        returns.insert(0, R)
    return returns

def update_model(observations, actions, returns, optimizer):
    """
    Perform a policy gradient update using a batch of training data.
    """
    observations = torch.stack(observations)
    actions = torch.tensor(actions)
    returns = torch.tensor(returns)

    optimizer.zero_grad()
    log_probs = torch.log(model(observations))
    selected_log_probs = log_probs[range(len(actions)), actions]
    loss = -torch.mean(selected_log_probs * returns)
    loss.backward()
    optimizer.step()


In [19]:

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        
        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, queries, mask):
        N = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        # Split the embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)

        queries = self.queries(queries)
        keys = self.keys(keys)
        values = self.values(values)

        # Scaled dot-product attention
        attention = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]) / self.embed_size**0.5
        if mask is not None:
            attention = attention.masked_fill(mask == 0, float("-inf"))

        attention = torch.nn.functional.softmax(attention, dim=3)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadSelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.LeakyReLU(0.05),  # Replaced ReLU with LeakyReLU
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.norm1(attention + query)
        x = self.dropout(x)
        forward = self.feed_forward(x)
        out = self.norm2(forward + x)
        out = self.dropout(out)
        return out

def positional_encoding(position, d_model):
    """
    Compute the positional encoding for a given position and model size.
    """
    def _calc_freq(position, i):
        return position / torch.pow(10000, 2 * (i // 2) / d_model)
    
    position = torch.arange(position).unsqueeze(1).float()
    div_term = torch.arange(0, d_model, 2).float()
    
    # Apply sin to even positions in the array; 2i
    pe = torch.zeros([position.shape[0], d_model])
    pe[:, 0::2] = torch.sin(_calc_freq(position, div_term))
    
    # Apply cos to odd positions in the array; 2i+1
    pe[:, 1::2] = torch.cos(_calc_freq(position, div_term))
    
    return pe

class GPT(nn.Module):
    def __init__(self, embed_size, num_layers, heads, forward_expansion, max_length, dropout):
        super(GPT, self).__init__()
        self.embed_size = embed_size
        self.model_type = 'GPT'
        self.classification_head = nn.Linear(embed_size, 1)
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )
        self.dropout = nn.Dropout(dropout)
        
        # Using the model's device
        device = next(self.parameters()).device
        self.pe = positional_encoding(max_length, embed_size).to(device)


    def forward(self, x, mask=None):
        N, seq_length, _ = x.shape
        
        out = self.dropout(x + self.pe[:x.size(1), :])  # Use sinusoidal positional encoding

        for layer in self.layers:
            out = layer(out, out, out, mask)

        # Take the output from the last token in the sequence
        seq_last_output = out[:, -1, :]
        logits = self.classification_head(seq_last_output)
        
        return logits
    

In [26]:
import torch
import torch.optim as optim
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

# Initialize the trading environment
env = TradingEnv(df_use)  # Assuming df is your DataFrame

# # Initialize the Transformer policy model
# input_dim = 3  # Adjust this based on your environment's observation space
# model = TransformerPolicy(input_dim=input_dim, d_model=64, nhead=2, num_layers=2, num_actions=3)

embed_size = 150  # Size of each embedding vector
num_layers = 2    # Number of Transformer blocks
heads = 5         # Number of attention heads
forward_expansion = 4
max_length = 50   # Max length of the sequence
dropout = 0.1

model = GPT(embed_size, num_layers, heads, forward_expansion, max_length, dropout).to(device)



model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.00001)


Using GPU: NVIDIA GeForce GTX 970


In [27]:
num_episodes = 100
gamma = 0.99  # Discount factor for future rewards
accumulation_steps = 100

for episode in range(num_episodes):
    observation = env.reset()
    observation = torch.FloatTensor(observation).to(device)  # Move observation to GPU
    done = False
    episode_rewards = []
    episode_observations = []
    episode_actions = []

    while not done:
        action = select_action(model, observation)
        next_observation, reward, done, _ = env.step(action)

        episode_rewards.append(reward)
        episode_observations.append(process_observation(observation))
        episode_actions.append(action)

        observation = next_observation

    returns = compute_returns(episode_rewards, gamma)

    optimizer.zero_grad()
    for i, (observation, action, return_) in enumerate(zip(episode_observations, episode_actions, returns)):
        log_probs = torch.log(model(observation.unsqueeze(0)))
        selected_log_probs = log_probs[0, action]
        loss = -selected_log_probs * return_
        loss.backward()

        if (i + 1) % accumulation_steps == 0 or i == len(episode_observations) - 1:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()

    total_reward = sum(episode_rewards)
    initial_budget = env.initial_budget
    current_budget = env.current_budget
    return_rate = (current_budget - initial_budget) / initial_budget * 100
    print(f"Episode {episode+1}, Total Reward: {total_reward}, Return {return_rate:.2f}%")


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.